In [ ]:
import os
import PIL
import pickle
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import Input, Reshape, Dense, Conv2D, Conv2DTranspose, Dropout, Flatten, ReLU, LeakyReLU, BatchNormalization, UpSampling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.preprocessing import image
from google.colab import drive

In [ ]:
drive.mount('/content/gdrive')

In [ ]:
#!mkdir '/content/landscape-dataset/'
#!unzip  '/content/gdrive/MyDrive/landscape dataset.zip' -d '/content/landscape-dataset/'

In [ ]:
PATH_TO_DATA = '/content/landscape-dataset/landscape dataset/'
IMAGE_SHAPE = (128,128,3)
FILENAMES = os.listdir(PATH_TO_DATA)
DATASET_SIZE = len(FILENAMES)

In [ ]:
def normalize(img):
    return (img / 127.5) - 1

def latent_points(size, latent_dim):  
    z_vector = np.random.normal(0,1, (size, latent_dim))
    return z_vector

def plot_generated_images(generator, n, latent_dim):
    latent = latent_points(n, latent_dim)
    imgs = (generator(latent) + 1) / 2

    _, axs = plt.subplots(1, n, figsize=(12, 12))
    axs = axs.flatten()

    for img, ax in zip(imgs, axs):
        ax.imshow(img)
    plt.show()

def load_data():
    images = np.empty((DATASET_SIZE, *IMAGE_SHAPE), dtype = np.float32)

    for i, filename in tqdm(enumerate(FILENAMES)):
        path = os.path.join(PATH_TO_DATA, filename)
        
        img = PIL.Image.open(path).convert('RGB').resize(IMAGE_SHAPE[:2], PIL.Image.ANTIALIAS)
        img = normalize(np.asarray(img))
        
        images[i, ...] = img
  
    return images

In [ ]:
train_images = load_data()

4319it [01:33, 46.06it/s]


In [ ]:
class DCGAN:
    def __init__(self):
        self.batch_size = 64
        self.epochs = 400
        self.latent_dim = 150
        self.weight_init = RandomNormal(stddev=0.02)
        self.alpha = 0.2
        self.generator_lr = 2e-4
        self.discriminator_lr = 2e-4       
        self.gen_optimizer = Adam(self.generator_lr, beta_1 = 0.5)
        self.disc_optimizer = Adam(self.discriminator_lr, beta_1 = 0.5)
        self.generator = self.get_generator()
        self.discriminator = self.get_discriminator()
        self.gan = self.adversarial_model(self.generator, self.discriminator)

    def generate_fake_images(self, size):
        latent_vectors = latent_points(size, self.latent_dim)
        images = self.generator(latent_vectors)
        return images

    def ConvBlock(self, input, filters, kernel_size, drop_rate = 0.3):
        x = Conv2D(filters = filters, kernel_size = kernel_size, padding = 'same', strides = 2, kernel_initializer=self.weight_init)(input)
        x = LeakyReLU(alpha = self.alpha)(x)
        x = Dropout(drop_rate)(x)
        return x

    def UpSamplingBlock(self, input, filters):
        x = UpSampling2D((2,2))(input)
        x = Conv2D(filters = filters, kernel_size = 4, strides = 1, padding = 'same', kernel_initializer = self.weight_init)(x)
        x = BatchNormalization(momentum = 0.8)(x, training = True)
        x = LeakyReLU(alpha = self.alpha)(x)
        return x

    def get_discriminator(self):
        input_layer = Input(IMAGE_SHAPE)

        x = self.ConvBlock(input_layer, filters = 64, kernel_size = 3)
        x = self.ConvBlock(x, filters = 128, kernel_size = 3)
        x = self.ConvBlock(x, filters = 256, kernel_size = 3)
        x = self.ConvBlock(x, filters = 512, kernel_size = 3)

        x = Flatten()(x)
        output = Dense(units = 1, activation='sigmoid')(x)

        discriminator = Model(inputs = input_layer, outputs = output)
        discriminator.compile(loss = 'binary_crossentropy', optimizer = self.disc_optimizer)
        return discriminator

    def get_generator(self):
        n_units = 8*8

        input_layer = Input((self.latent_dim, ))
        x = Dense(units = n_units * 128)(input_layer)
        x = BatchNormalization()(x, training = True)
        x = LeakyReLU(alpha = self.alpha)(x)
        x = Reshape((8,8,128))(x)

        x = self.UpSamplingBlock(x, filters = 64)
        x = self.UpSamplingBlock(x, filters = 128)
        x = self.UpSamplingBlock(x, filters = 128)
        x = self.UpSamplingBlock(x, filters = 256)

        output = Conv2D(filters = 3, kernel_size = 3, padding = 'same', activation = 'tanh')(x)

        generator = Model(inputs = input_layer, outputs = output)
        return generator

    def adversarial_model(self, generator, discriminator):
        discriminator.trainable = False

        gan = Sequential([
                generator,
                discriminator])

        gan.compile(loss = 'binary_crossentropy', optimizer = self.gen_optimizer)
        return gan

    def train(self):
        size = self.batch_size // 2
        k_steps = DATASET_SIZE // self.batch_size
        std = 0.1  # initial standard deviation value for noise decay

        for epoch in range(self.epochs):
          disc_loss_real = disc_loss_fake = generator_loss = 0
          noise_decay_rate = epoch/self.epochs
          
          new_std = std - noise_decay_rate
          new_std = new_std if new_std > 0 else 0

          for k in tqdm(range(k_steps)):
            real_imgs = train_images[k*size: (k+1)*size] + tf.random.normal(shape = IMAGE_SHAPE, mean=0, stddev=new_std, dtype=tf.float32)   # add gaussian noise only to the real images 
            real_imgs = tf.image.random_flip_left_right(real_imgs)
            fake_imgs = self.generate_fake_images(size)

            disc_loss_real = self.discriminator.train_on_batch(real_imgs, np.ones((size, 1)) * 0.9)    # label smoothing
            disc_loss_fake = self.discriminator.train_on_batch(fake_imgs, np.zeros((size, 1)) + 0.1)

            latent_space = latent_points(self.batch_size, self.latent_dim)
            generator_loss = self.gan.train_on_batch(latent_space, np.ones((self.batch_size, 1)) * 0.9)

          print(f'''Epoch: {epoch+1}/{self.epochs}           
          Discriminator loss on real images: {disc_loss_real}
          Discriminator loss on fake images: {disc_loss_fake}
          Generator loss: {generator_loss}''')

          # visualizing the results
          for _ in range(2):
            plot_generated_images(generator, n = 5, latent_dim = self.latent_dim)

In [ ]:
dcgan = DCGAN()
generator = dcgan.generator
#discriminator = dcgan.discriminator
#gan = dcgan.gan

In [ ]:
generator.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 150)]             0         
_________________________________________________________________
dense (Dense)                (None, 8192)              1236992   
_________________________________________________________________
batch_normalization (BatchNo (None, 8192)              32768     
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 8192)              0         
_________________________________________________________________
reshape (Reshape)            (None, 8, 8, 128)         0         
_________________________________________________________________
up_sampling2d (UpSampling2D) (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 16, 16, 64)        131136

In [ ]:
dcgan.train()

In [ ]:
for _ in range(20):
  plot_generated_images(generator, n = 3, latent_dim = 150)